In [1]:
import math
import pandas as pd
from datetime import datetime

In [2]:
user_ids = [1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 37, 38, 39, 40, 44, 45, 46, 47, 48]
len(user_ids)

35

In [3]:
db = '/Users/shiyecao/Desktop/AI-Influence/data-analysis-2/'

# Gaze Data
fixation_df = pd.read_csv(db+'fixation_bucket_0.5.csv', sep=',')
shift_df = pd.read_csv(db+'bucket_gaze_shift_less_0.5.csv', sep=',')
duration_df = pd.read_csv(db+'bucket_gaze_duration_divided_by_fixation_total_0.5.csv', sep=',')

# Response Data
user = pd.read_excel(db + "user.xlsx", sep="") 
demographic = pd.read_excel(db + "demographic.xlsx", sep="") 
practice = pd.read_excel(db + "practice.xlsx", sep="")

main1 = pd.read_excel(db + "main1.xlsx", sep="")
main2 = pd.read_excel(db + "main2.xlsx", sep="")
main3 = pd.read_excel(db + "main3.xlsx", sep="")
main = main1.append(main2)
main = main.append(main3)

time1 = pd.read_excel(db + "time1.xlsx", sep="")
time2 = pd.read_excel(db + "time2.xlsx", sep="")
time3 = pd.read_excel(db + "time3.xlsx", sep="")
time = time1.append(time2)
time = time.append(time3)

# Survey Data
practice_survey_one = pd.read_excel(db + 'practice_survey_one.xlsx', sep="")
practice_survey_two = pd.read_excel(db + 'practice_survey_two.xlsx', sep="")
practice_survey_three = pd.read_excel(db + 'practice_survey_three.xlsx', sep="")
final_survey = pd.read_excel(db + 'final_survey.xlsx', sep="")

middle_survey1 = pd.read_excel(db + 'middle_survey1.xlsx', sep="")
middle_survey2 = pd.read_excel(db + 'middle_survey2.xlsx', sep="")
middle_survey3 = pd.read_excel(db + 'middle_survey3.xlsx', sep="")
middle_survey = middle_survey1.append(middle_survey2)
middle_survey = middle_survey.append(middle_survey3)

In [4]:
ending_survey = middle_survey.loc[middle_survey['Round'] == 10]
ending_survey = ending_survey.rename({'Q1': 'survey_reliance'}, axis=1)
ending_survey = ending_survey.rename({'Q2': 'survey_trust'}, axis=1)
ending_survey = ending_survey.rename({'Q3': 'survey_confidence'}, axis=1)
ending_survey = ending_survey.rename({'Q4': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.loc[middle_survey['Round'] != 10]
middle_survey = middle_survey.rename({'Q2': 'survey_reliance'}, axis=1)
middle_survey = middle_survey.rename({'Q3': 'survey_trust'}, axis=1)
middle_survey = middle_survey.rename({'Q4': 'survey_confidence'}, axis=1)
middle_survey = middle_survey.rename({'Q1': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.append(ending_survey)

shift_df_merged = shift_df.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])
duration_df_merged = duration_df.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])
fixation_df_merged = fixation_df.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])

In [5]:
shift_df_merged.to_csv(db+'bucket_gaze_shift_less_0.5_merged.csv', index=False) 
duration_df_merged.to_csv(db+'bucket_gaze_duration_divided_by_fixation_total_0.5_merged.csv', index=False)
fixation_df_merged.to_csv(db+'fixation_backet_0.5_merged.csv', index=False)

In [6]:
user_acc = user[['User id', 'Acc']].groupby('User id', as_index=False).first()
orientation_df = main.groupby(['User id', 'Round'], as_index=False).first()[['User id', 'Round', 'Orientation']]

shift_less = shift_df_merged.groupby(['User id', 'round', 'shift'], as_index=False)[['percent_shift', 'shift_per_sec']].sum()
shift_less_merged = shift_less.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])
shift_less_merged = shift_less_merged.merge(orientation_df, left_on=["User id", "round"], right_on=["User id", "Round"])
shift_less_merged = shift_less_merged.merge(user_acc, left_on=["User id"], right_on=["User id"])
shift_less_merged.to_csv(db+'gaze_shift_less.csv', index=False) 

duration = duration_df_merged.groupby(['User id', 'round', 'location'], as_index=False)[['percent duration']].sum()
duration_merged = duration.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])
duration_merged = duration_merged.merge(orientation_df, left_on=["User id", "round"], right_on=["User id", "Round"])
duration_merged = duration_merged.merge(user_acc, left_on=["User id"], right_on=["User id"])
duration_merged.to_csv(db+'gaze_duration.csv', index=False) 

fixation = fixation_df_merged.groupby(['User id', 'round', 'location'], as_index=False).first()
fixation_merged = fixation.merge(middle_survey, left_on=["User id", "round"], right_on=["User id", "Round"])
fixation_merged = fixation_merged.merge(orientation_df, left_on=["User id", "round"], right_on=["User id", "Round"])
fixation_merged = fixation_merged.merge(user_acc, left_on=["User id"], right_on=["User id"])
fixation_merged.to_csv(db+'first_fixation.csv', index=False) 

In [18]:
dur_df = duration_df_merged.groupby(['User id', 'round', 'location'], as_index=False)[['percent duration']].sum()
df = duration_df_merged[['User id', 'round', 'orientation', 'Acc', 'survey_agreement',
       'survey_reliance', 'survey_trust', 'survey_confidence']].groupby(['User id', 'round'], as_index=False).first()
duration_merged = dur_df.merge(df, left_on=["User id", "round"], right_on=["User id", "round"])

duration_merged.to_csv(db+'gaze_duration_merged.csv', index=False)

In [ ]:
shift_df_merged.to_csv(db+'bucket_gaze_shift_less_0.5_merged.csv', index=False) 
duration_df_merged.to_csv(db+'bucket_gaze_duration_divided_by_fixation_total_0.5_merged.csv', index=False)
fixation_df_merged.to_csv(db+'fixation_backet_0.5_merged.csv', index=False)

In [51]:
shift_less = shift_df.groupby(['User id', 'round', 'shift'], as_index=False)[['percent_shift', 'shift_per_sec']].sum()
shift_less

,User id,round,shift,percent_shift,shift_per_sec
0,1,1,2AI,0.566667,0.470588
1,1,1,2Task,0.933333,0.823529
2,1,1,2User,0.500000,0.705882
3,1,2,2AI,0.115996,0.080808
4,1,2,2Task,0.967426,0.592593
...,...,...,...,...,...
1036,48,9,2Task,0.852941,1.043478
1037,48,9,2User,0.970588,1.217391
1038,48,10,2AI,0.267857,0.270270
1039,48,10,2Task,0.886905,0.918919


# Participant Demographics

In [10]:
all_user_ids = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 44, 45, 46, 47, 48]
len(all_user_ids)

42

In [11]:
demographic.loc[demographic['User id'].isin(all_user_ids)][['Age', 'Gender','Education', 'Familiarity', 'Trust']].describe()

,Age,Education,Familiarity,Trust
count,42.000000,42.000000,42.00000,42.000000
mean,23.690476,3.047619,3.47619,3.285714
std,6.134620,1.267846,0.94322,0.708338
min,19.000000,1.000000,1.00000,2.000000
25%,21.000000,2.000000,3.00000,3.000000
50%,22.000000,3.000000,3.00000,3.000000
75%,23.750000,3.000000,4.00000,4.000000
max,60.000000,6.000000,5.00000,4.000000


In [12]:
demographic.loc[demographic['User id'].isin(all_user_ids)]['Gender'].value_counts()

male      22
female    20
Name: Gender, dtype: int64

In [4]:
demographic.loc[demographic['User id'].isin(user_ids)][['Age', 'Gender','Education', 'Familiarity', 'Trust']].describe()

,Age,Education,Familiarity,Trust
count,35.000000,35.000000,35.000000,35.000000
mean,22.457143,2.828571,3.371429,3.228571
std,1.868357,1.124218,0.942016,0.731063
min,19.000000,1.000000,1.000000,2.000000
25%,21.000000,2.000000,3.000000,3.000000
50%,22.000000,3.000000,3.000000,3.000000
75%,23.000000,3.000000,4.000000,4.000000
max,27.000000,5.000000,5.000000,4.000000


In [7]:
demographic['Gender'].value_counts()

female    25
male      23
Name: Gender, dtype: int64

In [34]:
user

,User id,Q order,P order,Acc,Round Num,Start Time,End Time
0,11,"5,2,7,6,9,4,1,8,3,0","1,0",1,10,2021-11-12 22:07:09.010447+00,2021-11-12 22:29:33.776449+00
1,25,"6,7,0,1,8,3,2,9,4,5","0,1",0,10,2021-11-16 16:07:48.662727+00,2021-11-16 16:29:39.453035+00
2,21,"5,2,7,8,9,6,3,0,1,4","1,0",2,10,2021-11-15 21:05:07.301656+00,2021-11-15 21:41:02.155368+00
3,6,"0,9,2,7,6,5,4,1,8,3","0,1",2,10,2021-11-12 16:03:39.818005+00,2021-11-12 16:22:31.549742+00
4,1,"6,3,2,1,8,5,0,9,4,7","1,0",1,10,2021-11-09 01:17:21.248339+00,2021-11-09 01:40:28.506122+00
5,37,"3,6,5,0,7,4,9,8,1,2","1,0",0,10,2021-12-08 20:39:07.787453+00,2021-12-08 21:06:56.626682+00
6,17,"6,5,4,3,8,9,0,7,2,1","0,1",0,10,2021-11-15 00:10:56.672703+00,2021-11-15 00:39:41.009562+00
7,12,"0,3,2,7,6,9,4,5,8,1","0,1",1,10,2021-11-13 20:55:01.76381+00,2021-11-13 21:24:23.092011+00
8,32,"9,6,3,2,1,8,5,4,7,0","1,0",0,10,2021-11-17 20:13:38.529837+00,2021-11-17 20:30:22.339331+00
9,7,"3,4,5,6,7,0,1,8,9,2","0,1",0,10,2021-11-12 17:44:11.179162+00,2021-11-12 18:15:32.640151+00


In [45]:
user.iloc[1]['Start Time'][0:19]

'2021-11-16 16:07:48'

In [53]:
user = user.loc[user['User id'].isin(all_user_ids)]
user['start_time'] = user.apply(lambda x: datetime.strptime(x['Start Time'][0:19], '%Y-%m-%d %H:%M:%S'), axis=1)
user['end_time'] = user.apply(lambda x: datetime.strptime(x['End Time'][0:19], '%Y-%m-%d %H:%M:%S'), axis=1)
user['length'] = user['end_time'] - user['start_time']
user['length'].describe()

count                        42
mean     0 days 00:25:28.238095
std      0 days 00:07:53.881008
min             0 days 00:10:15
25%             0 days 00:20:05
50%      0 days 00:23:38.500000
75%      0 days 00:29:30.250000
max             0 days 00:49:38
Name: length, dtype: object

# Manipulation Check 

In [20]:
practice = practice.loc[practice['User id'].isin(user_ids)]
practice_survey_one = practice_survey_one.rename({'Q1': 'practice_survey_difficulty'}, axis=1) 
practice_survey_one['Round'] = 1
practice_survey_two = practice_survey_two.rename({'Q1': 'practice_survey_difficulty'}, axis=1) 
practice_survey_two['Round'] = 2
practice_survey = practice_survey_one.append(practice_survey_two)
practice = practice.merge(practice_survey[['User id', 'practice_survey_difficulty', 'Round']], 
                                 left_on=["User id", "Round"], right_on=["User id", "Round"])

In [21]:
practice_acc_df = pd.DataFrame(columns=['User id', 'Round','Acc'], dtype=object)

for u in range(len(practice)):
    user_id = practice.iloc[u]['User id']
    roundId = practice.iloc[u]['Round']
    
    resp = practice.iloc[u]['User Response'].split(",")
    truth = practice.iloc[u]['Truth'].split(",")
    
    count = 0
    for i in range(len(resp)): 
        if resp[i] == truth[i]:
            count += 1

    practice_acc = count/16
    
    practice_acc_df = practice_acc_df.append({'User id': user_id, 'Round': roundId, 'Acc': practice_acc}, ignore_index=True)

practice = practice.merge(practice_acc_df, left_on=["User id", "Round"], right_on=["User id", "Round"])

In [22]:
practice.loc[practice['Orientation'] == 'top'][['practice_survey_difficulty', 'Acc']].describe()

,practice_survey_difficulty,Acc
count,35.000000,35.000000
mean,3.171429,0.989286
std,1.294461,0.053563
min,1.000000,0.687500
25%,2.000000,1.000000
50%,3.000000,1.000000
75%,4.000000,1.000000
max,7.000000,1.000000


In [23]:
practice.loc[practice['Orientation'] == 'bottom'][['practice_survey_difficulty', 'Acc']].describe()

,practice_survey_difficulty,Acc
count,35.000000,35.000000
mean,4.942857,0.485714
std,1.258918,0.239238
min,3.000000,0.250000
25%,4.000000,0.375000
50%,5.000000,0.375000
75%,6.000000,0.437500
max,7.000000,1.000000


In [24]:
practice.to_csv(db+'practice_merged.csv', index=False) 

In [25]:
practice

,Id,Q id,User id,Round,User Response,Truth,Orientation,Num Changes,Num Reset,Response Time,Reset Time,practice_survey_difficulty,Acc
0,1,p1,1,1,"0,0,1,1,0,0,3,0,4,1,3,2,0,2,4,0","0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0",bottom,9,0,111,NaN,5,0.375
1,2,p0,1,2,"0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1","0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1",top,8,0,45,NaN,3,1.000
2,3,p0,2,1,"0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1","0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1",top,8,0,56,NaN,4,1.000
3,4,p1,2,2,"0,0,1,1,0,2,3,0,4,1,3,3,0,2,4,0","0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0",bottom,11,0,141,NaN,6,0.375
4,7,p1,4,1,"0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0","0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0",bottom,8,0,56,NaN,4,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,91,p0,46,2,"0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1","0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1",top,8,0,79,NaN,1,1.000
66,92,p0,47,1,"0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1","0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1",top,9,0,43,NaN,1,1.000
67,93,p1,47,2,"0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0","0,2,0,1,4,3,3,1,2,1,0,0,0,4,0,0",bottom,8,0,88,NaN,4,1.000
68,94,p0,48,1,"0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1","0,4,4,2,0,2,4,1,0,3,0,4,0,0,0,1",top,8,0,41,NaN,3,1.000


In [26]:
main = main.loc[main['User id'].isin(user_ids)]
ending_survey = middle_survey.loc[middle_survey['Round'] == 10]
ending_survey = ending_survey.rename({'Q1': 'survey_reliance'}, axis=1)
ending_survey = ending_survey.rename({'Q2': 'survey_trust'}, axis=1)
ending_survey = ending_survey.rename({'Q3': 'survey_confidence'}, axis=1)
ending_survey = ending_survey.rename({'Q4': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.loc[middle_survey['Round'] != 10]
middle_survey = middle_survey.rename({'Q2': 'survey_reliance'}, axis=1)
middle_survey = middle_survey.rename({'Q3': 'survey_trust'}, axis=1)
middle_survey = middle_survey.rename({'Q4': 'survey_confidence'}, axis=1)
middle_survey = middle_survey.rename({'Q1': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.append(ending_survey)
main = main.merge(middle_survey, left_on=["User id", "Round"], right_on=["User id", "Round"])

In [27]:
main_acc_df = pd.DataFrame(columns=['User id', 'Round', 'final_user_ai_acc', 'final_user_truth_acc', 'ai_truth_acc', 'start_user_ai_acc', 'start_user_truth_acc'], dtype=object)

for u in range(len(main)):
    user_id = main.iloc[u]['User id']
    roundnum = main.iloc[u]['Round']
    start_resp = main.iloc[u]['Before AIresponse'].split(",")
    final_resp = main.iloc[u]['User Response'].split(",")
    truth = main.iloc[u]['Truth'].split(",")
    ai = main.iloc[u]['Ai Response'][1:-1].split(",")
    orientation = main.iloc[u]['Orientation']
    
    count_final_user_ai_diff=0
    count_final_user_truth_diff=0
    count_ai_truth_diff=0
    
    count_start_user_ai_diff=0
    count_start_user_truth_diff=0
    
    for i in range(len(resp)): 
        if final_resp[i] == ai[i]:
            count_final_user_ai_diff += 1
        
        if final_resp[i] == truth[i]:
            count_final_user_truth_diff += 1
            
        if ai[i] == truth[i]:
            count_ai_truth_diff += 1
        
        if start_resp[i] == ai[i]:
            count_start_user_ai_diff += 1
        
        if start_resp[i] == truth[i]:
            count_start_user_truth_diff += 1
            
    main_acc_df = main_acc_df.append({'User id': user_id, 'Round': roundnum, 'final_user_ai_acc': count_final_user_ai_diff/16, 
                                      'final_user_truth_acc': count_final_user_truth_diff/16, 'ai_truth_acc': count_ai_truth_diff/16,
                                      'start_user_ai_acc': count_start_user_ai_diff/16, 'start_user_truth_acc': count_start_user_truth_diff/16,}, ignore_index=True)
    
main = main.merge(main_acc_df, left_on=["User id", "Round"], right_on=["User id", "Round"])

In [55]:
duration_main_merged = duration_merged[['User id', 'round', 'location', 'percent duration']].merge(main, left_on=["User id", "round"], right_on=["User id", "Round"])

In [56]:
duration_main_merged['start_end_user_ai_diff'] = duration_main_merged['final_user_ai_acc'] - duration_main_merged['start_user_ai_acc']

In [45]:
duration_main_merged.to_csv(db+'duration_main_merged.csv', index=False) 

In [15]:
main.loc[main['User id'].isin(user_ids)]

,Id_x,Q id,User id,Round,User Response,Before AIresponse,Ai Response,Ai Acc,Truth,Orientation,...,Id_y,survey_agreement,survey_reliance,survey_trust,survey_confidence,final_user_ai_acc,final_user_truth_acc,ai_truth_acc,start_user_ai_acc,start_user_truth_acc
0,1,t6,1,1,"0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0","0,4,2,1,0,3,0,1,0,0,0,0,0,0,0,0","{0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0}",1,"0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0",top,...,1,6,5,5,6,1.0000,1.0000,1.0000,0.6250,0.6250
1,2,t3,1,2,"0,0,0,2,1,3,3,1,0,2,2,1,0,4,0,0","0,0,0,2,0,0,2,2,0,0,2,0,0,0,0,0","{0,0,1,2,0,2,2,3,0,4,0,3,0,0,0,1}",1,"0,0,1,2,0,2,2,3,0,4,1,3,0,0,0,1",bottom,...,2,1,1,1,2,0.3750,0.3750,0.9375,0.5625,0.5625
2,3,t2,1,3,"0,0,1,2,0,0,1,3,0,0,3,0,0,1,3,1","0,0,1,2,0,0,1,3,0,0,1,0,0,0,0,0","{0,0,1,2,0,0,1,3,0,0,3,0,0,1,3,1}",1,"0,0,1,2,0,0,1,3,0,1,3,0,0,1,3,1",top,...,3,6,6,6,6,1.0000,0.9375,0.9375,0.7500,0.6875
3,4,t1,1,4,"0,0,2,4,0,3,0,2,0,2,0,1,0,1,2,3","0,0,2,4,0,0,0,2,0,0,0,1,0,0,0,1","{3,1,2,4,2,0,0,2,1,2,3,0,0,0,0,0}",1,"3,1,2,4,2,0,0,2,1,2,3,0,0,0,0,0",bottom,...,4,1,1,1,6,0.3750,0.3750,1.0000,0.5000,0.5000
4,5,t8,1,5,"0,3,2,2,0,4,0,1,1,3,1,4,0,0,0,0","0,3,2,2,0,0,0,1,0,0,0,1,0,0,0,0","{0,3,2,2,0,4,0,1,1,3,1,4,0,0,0,0}",1,"0,3,2,2,0,4,0,1,1,3,1,4,0,0,0,0",top,...,5,7,7,7,7,1.0000,1.0000,1.0000,0.6875,0.6875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,456,t1,48,6,"3,1,2,4,2,0,0,2,1,2,3,0,0,0,0,0","2,4,2,4,0,0,0,0,0,0,0,0,0,0,0,0","{3,1,2,4,2,0,0,2,1,2,3,0,0,0,0,0}",1,"3,1,2,4,2,0,0,2,1,2,3,0,0,0,0,0",bottom,...,456,6,6,5,6,1.0000,1.0000,1.0000,0.5625,0.5625
346,457,t0,48,7,"3,2,4,4,0,0,0,2,0,2,2,1,0,0,2,0","0,0,0,4,0,0,0,2,0,1,2,1,0,0,0,0","{3,2,4,4,0,0,0,2,0,2,2,1,0,0,2,2}",1,"3,2,4,4,0,0,0,2,0,2,2,1,0,0,2,0",top,...,457,2,2,2,6,0.9375,1.0000,0.9375,0.6250,0.6875
347,458,t7,48,8,"1,2,1,4,2,1,2,0,1,0,2,0,0,0,0,0","1,2,1,4,1,0,0,0,0,0,0,0,0,0,0,0","{1,2,1,4,2,1,2,1,1,0,2,0,0,0,0,0}",1,"1,2,1,4,2,1,2,0,1,0,2,0,0,0,0,0",bottom,...,458,2,2,2,6,0.9375,1.0000,0.9375,0.6250,0.6875
348,459,t6,48,9,"0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0","0,4,2,1,0,3,1,0,0,0,0,0,0,0,0,0","{0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0}",1,"0,4,2,1,0,3,2,0,1,2,3,0,0,4,0,0",top,...,459,6,2,6,6,1.0000,1.0000,1.0000,0.6875,0.6875


In [57]:
duration_main_merged = duration_main_merged.loc[duration_main_merged['location'] == 'AI']

In [58]:
duration_main_merged.to_csv(db+'duration_main_merged.csv', index=False) 

In [62]:
duration_main_merged = duration_main_merged[['User id', 'round', 'location', 'percent duration', 
                      'Ai Acc', 'Orientation', 'survey_agreement', 'survey_reliance', 'survey_trust',
                      'survey_confidence', 'final_user_ai_acc', 'final_user_truth_acc',
                      'ai_truth_acc', 'start_user_ai_acc', 'start_user_truth_acc',
                      'start_end_user_ai_diff']]

duration_main_merged.to_csv(db+'duration_main_merged.csv', index=False) 

In [64]:
duration_main_merged['percent duration'] = duration_main_merged['percent duration'] / 2

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
duration_main_merged

,User id,round,location,percent duration,Ai Acc,Orientation,survey_agreement,survey_reliance,survey_trust,survey_confidence,final_user_ai_acc,final_user_truth_acc,ai_truth_acc,start_user_ai_acc,start_user_truth_acc,start_end_user_ai_diff
0,1,1,AI,0.175703,1,top,6,5,5,6,1.0000,1.0000,1.0000,0.6250,0.6250,0.3750
3,1,2,AI,0.061107,1,bottom,1,1,1,2,0.3750,0.3750,0.9375,0.5625,0.5625,-0.1875
6,1,3,AI,0.361498,1,top,6,6,6,6,1.0000,0.9375,0.9375,0.7500,0.6875,0.2500
9,1,4,AI,0.139291,1,bottom,1,1,1,6,0.3750,0.3750,1.0000,0.5000,0.5000,-0.1250
12,1,5,AI,0.191397,1,top,7,7,7,7,1.0000,1.0000,1.0000,0.6875,0.6875,0.3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,48,6,AI,0.130683,1,bottom,6,6,5,6,1.0000,1.0000,1.0000,0.5625,0.5625,0.4375
1029,48,7,AI,0.078584,1,top,2,2,2,6,0.9375,1.0000,0.9375,0.6250,0.6875,0.3125
1032,48,8,AI,0.193513,1,bottom,2,2,2,6,0.9375,1.0000,0.9375,0.6250,0.6875,0.3125
1035,48,9,AI,0.079815,1,top,6,2,6,6,1.0000,1.0000,1.0000,0.6875,0.6875,0.3125


In [66]:
duration_main_merged.to_csv(db+'duration_main_merged.csv', index=False) 

In [67]:
duration_df = pd.read_csv(db+'bucket_gaze_duration_divided_by_fixation_total_0.5.csv', sep=',')

In [73]:
duration_df.columns

Index(['User id', 'round', 'bucket', 'location', 'percent duration',
       'orientation', 'FPOGD', 'Ai Acc'],
      dtype='object')

In [74]:
duration_df = duration_df.loc[duration_df['location'] == 'AI']
duration_df = duration_df.rename({'Acc': 'Ai Acc'}, axis=1)
duration_df[['User id', 'round', 'bucket', 'location', 'percent duration',
       'orientation', 'Ai Acc']].to_csv(db+'duration.csv', index=False) 

In [79]:
demographic.describe()

,User id,Age,Education,Familiarity,Trust
count,48.00,48.000000,48.000000,48.000000,48.000000
mean,24.50,25.645833,3.125000,3.375000,3.333333
std,14.00,9.822877,1.298526,1.023656,0.694456
min,1.00,19.000000,1.000000,1.000000,2.000000
25%,12.75,21.000000,2.000000,3.000000,3.000000
50%,24.50,22.000000,3.000000,3.000000,3.000000
75%,36.25,26.000000,3.500000,4.000000,4.000000
max,48.00,60.000000,6.000000,5.000000,4.000000


In [81]:
demographic['Familiarity'].mode()

0    3
dtype: int64

In [82]:
demographic['Trust'].mode()

0    4
dtype: int64

In [83]:
demographic['Trust'].median()

3.0